In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import kagglehub

In [ ]:
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")
print("Path to dataset files:", path)

dataset_dir = os.path.join(path, "data")
print("Contents of dataset_dir:", os.listdir(dataset_dir))

train_dir = os.path.join(dataset_dir, "train")
classes = sorted(os.listdir(train_dir))  
N_Wheat = len(classes)
print(f"Number of classes (N_Wheat): {N_Wheat}")
print("Classes:", classes)

datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    train_dir,  
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

valid_dir = os.path.join(dataset_dir, "valid")
validation_generator = datagen.flow_from_directory(
    valid_dir,  
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")

Using Colab cache for faster access to the 'wheat-plant-diseases' dataset.
Path to dataset files: /kaggle/input/wheat-plant-diseases
Contents of dataset_dir: ['valid', 'test', 'train']
Number of classes (N_Wheat): 15
Classes: ['Aphid', 'Black Rust', 'Blast', 'Brown Rust', 'Common Root Rot', 'Fusarium Head Blight', 'Healthy', 'Leaf Blight', 'Mildew', 'Mite', 'Septoria', 'Smut', 'Stem fly', 'Tan spot', 'Yellow Rust']
Found 13104 images belonging to 15 classes.
Found 300 images belonging to 15 classes.
Training samples: 13104
Validation samples: 300


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)  
output = Dense(N_Wheat, activation='softmax')(x)  
model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,068,786 (15.52 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [10]:
# Callbacks
checkpoint = ModelCheckpoint(
    'efficientnet_stage1_wheat.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,  # Stop if no improvement for 3 epochs
    mode='max',
    verbose=1
)

# Train
epochs = 5  # Adjust as needed
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[checkpoint, early_stop]
)

print("Training complete. Best model saved as 'efficientnet_stage1_wheat.h5'")

Epoch 1/5
410/410 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4230 - loss: 1.8882
Epoch 1: val_accuracy improved from -inf to 0.54667, saving model to efficientnet_stage1_wheat.h5


410/410 ━━━━━━━━━━━━━━━━━━━━ 1156s 3s/step - accuracy: 0.4233 - loss: 1.8872 - val_accuracy: 0.5467 - val_loss: 1.6073
Epoch 2/5
410/410 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6505 - loss: 1.1121
Epoch 2: val_accuracy improved from 0.54667 to 0.59667, saving model to efficientnet_stage1_wheat.h5


410/410 ━━━━━━━━━━━━━━━━━━━━ 1125s 3s/step - accuracy: 0.6505 - loss: 1.1120 - val_accuracy: 0.5967 - val_loss: 1.4608
Epoch 3/5
410/410 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6861 - loss: 0.9847
Epoch 3: val_accuracy improved from 0.59667 to 0.61000, saving model to efficientnet_stage1_wheat.h5


410/410 ━━━━━━━━━━━━━━━━━━━━ 1132s 3s/step - accuracy: 0.6861 - loss: 0.9847 - val_accuracy: 0.6100 - val_loss: 1.5065
Epoch 4/5
410/410 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6940 - loss: 0.9495
Epoch 4: val_accuracy improved from 0.61000 to 0.63333, saving model to efficientnet_stage1_wheat.h5


410/410 ━━━━━━━━━━━━━━━━━━━━ 1125s 3s/step - accuracy: 0.6940 - loss: 0.9495 - val_accuracy: 0.6333 - val_loss: 1.4504
Epoch 5/5
410/410 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7058 - loss: 0.9181
Epoch 5: val_accuracy did not improve from 0.63333
410/410 ━━━━━━━━━━━━━━━━━━━━ 1121s 3s/step - accuracy: 0.7058 - loss: 0.9181 - val_accuracy: 0.6300 - val_loss: 1.4240
Training complete. Best model saved as 'efficientnet_stage1_wheat.h5'


In [11]:
from tensorflow.keras.models import load_model
best_model = load_model('efficientnet_stage1_wheat.h5')
val_loss, val_acc = best_model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.5593 - loss: 1.4462
Validation Accuracy: 0.6033
